In [1]:
import math
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import numpy as np
import time
import statistics as st
from scipy.spatial import distance as dt
import distance
from operator import truediv, add, mul
from sklearn.preprocessing import MinMaxScaler

### Data loading

In [2]:
def load_Data(name):
    try:
        # TRAIN data
        # load fingerprints
        df_trnrss = pd.read_csv('data/'+ name + '_trnrss.csv', header=None).values.tolist()
        # load coordinates
        df_trncrd = pd.read_csv('data/' + name + '_trncrd.csv', header=None)

        # for now, drop columns of 'floor' and 'building', keep x,y,z
        df_trncrd.drop(df_trncrd.columns[[3,4]], axis=1, inplace=True)
        df_trncrd = df_trncrd.values.tolist()

        # TEST data
        # load fingerprints
        df_tstrss = pd.read_csv('data/' + name + '_tstrss.csv', header=None).values.tolist()
        # load coordinates
        df_tstcrd = pd.read_csv('data/' + name + '_tstcrd.csv', header=None)

        # for now, drop columns of 'floor' and 'building', keep x,y,z
        df_tstcrd.drop(df_tstcrd.columns[[3,4]], axis=1, inplace=True)
        df_tstcrd = df_tstcrd.values.tolist()

        return(df_trnrss, df_trncrd, df_tstrss, df_tstcrd)
    except:
        print("Could not open file")
        return

### Get neighbors

In [54]:
# all distance metrics accepted
distance_metrics = ["euclidean", "minkowskiL1", "minkowskiL2", "minkowskiL3", "minkowskiL4", "minkowskiL5", "cityblock",
                    "chebyshev", "sorensen", "gower", "soergel", "kulczynski_d", "lorentzian", "canberra", "intersection",
                    "wavehedges", "czekanowski_s", "czekanowski_d", "motyka_s", "motyka_d", "kulczynski_s", "ruzicka",
                    "tanimoto", "inner", "harmonic", "cosine", "kumar", "jaccard_s", "jaccard_d", "dice_s", "dice_d",
                    "fidelity_abs", "fidelity_incr", "fidelity_norm", "bhattacharrya_abs", "bhattacharrya_incr", "bhattacharrya_norm", "hellinger", "matusita", "squared_chord", "sqeuclidean", "pearson",
                    "neyman", "squared", "prob_sym", "divergence", "clark", "additive_sym", "kullback_PQ", "jeffreys",
                    "k_divergence", "topsoe_abs", "topsoe_incr", "topsoe_norm", "jensen_shannon", "jensen_diff", "taneja", "kumar_johnson", "avgL",
                    "vicis_wave_hedges", "vicis_symm1", "vicis_symm2", "vicis_symm3", "min_symm", "max_symm"]

In [56]:
# Locate the most similar neighbors and return list of indexes
def get_neighbors(train, test_row, k, name):
    distances = list()
    for idx, train_row in enumerate(train):
        # Lp Minkowski family
        if name == "euclidean": dist = math.dist(test_row, train_row)
        if name == "minkowskiL1": dist = dt.minkowski(test_row, train_row, 1)
        if name == "minkowskiL2": dist = dt.minkowski(test_row, train_row, 2)
        if name == "minkowskiL3": dist = dt.minkowski(test_row, train_row, 3)
        if name == "minkowskiL4": dist = dt.minkowski(test_row, train_row, 4)
        if name == "minkowskiL5": dist = dt.minkowski(test_row, train_row, 5)
        if name == "cityblock": dist = dt.cityblock(test_row, train_row)
        if name == "chebyshev": dist = dt.chebyshev(test_row, train_row)

        # L1 family
        if name == "sorensen": dist = distance.sorensen(test_row, train_row)
        if name == "gower": dist = gower(test_row, train_row)
        if name == "soergel": dist = soergel(test_row, train_row)
        if name == "kulczynski_d": dist = kulczynski_d(test_row, train_row)
        if name == "lorentzian": dist = lorentzian(test_row, train_row)
        if name == "canberra": dist = dt.canberra(test_row, train_row)

        # Intersection family
        if name == "intersection": dist = intersection(test_row, train_row)
        if name == "wavehedges": dist = wavehedges(test_row, train_row)
        if name == "czekanowski_s": dist = czekanowski_s(test_row, train_row)
        if name == "czekanowski_d": dist = czekanowski_d(test_row, train_row)
        if name == "motyka_s": dist = motyka_s(test_row, train_row)
        if name == "motyka_d": dist = motyka_d(test_row, train_row)
        if name == "kulczynski_s": dist = kulczynski_s(test_row, train_row)
        if name == "ruzicka": dist = ruzicka(test_row, train_row)
        if name == "tanimoto": dist = tanimoto(test_row, train_row)

        # Inner product family
        if name == "inner": dist = np.inner(test_row, train_row)
        if name == "harmonic": dist = harmonic(test_row, train_row)
        if name == "cosine": dist = dt.cosine(test_row, train_row)
        if name == "kumar": dist = kumar(test_row, train_row)
        if name == "jaccard_s": dist = jaccard_s(test_row, train_row)
        if name == "jaccard_d": dist = jaccard_d(test_row, train_row)
        if name == "dice_s": dist = dice_s(test_row, train_row)
        if name == "dice_d": dist = dice_d(test_row, train_row)

        # Fidelity family or Squared-chord family
        if name == "fidelity": dist = fidelity(test_row, train_row)
        if name == "fidelity_abs": dist = fidelity_abs(test_row, train_row)
        if name == "fidelity_incr": dist = fidelity_incr(test_row, train_row)
        if name == "fidelity_norm": dist = fidelity_norm(test_row, train_row)
        if name == "bhattacharrya_abs": dist = bhattacharrya_abs(test_row, train_row)
        if name == "bhattacharrya_incr": dist = bhattacharrya_incr(test_row, train_row)
        if name == "bhattacharrya_norm": dist = bhattacharrya_norm(test_row, train_row)
        if name == "hellinger": dist = hellinger(test_row, train_row)
        if name == "matusita": dist = matusita(test_row, train_row)
        if name == "squared_chord": dist = squared_chord(test_row, train_row)

        # Squared L2 family
        if name == "sqeuclidean": dist = dt.sqeuclidean(test_row, train_row)
        if name == "pearson": dist = pearson(test_row, train_row)
        if name == "neyman": dist = neyman(test_row, train_row)
        if name == "squared": dist = squared(test_row, train_row)
        if name == "prob_sym": dist = prob_sym(test_row, train_row)
        if name == "divergence": dist = divergence(test_row, train_row)
        if name == "clark": dist = clark(test_row, train_row)
        if name == "additive_sym": dist = additive_sym(test_row, train_row)

        # Shannon's entropy family
        if name == "kullback_PQ": dist = kullback_PQ(test_row, train_row)
        if name == "jeffreys": dist = jeffreys(test_row, train_row)
        if name == "k_divergence": dist = k_divergence(test_row, train_row)
        if name == "topsoe_abs": dist = topsoe_abs(test_row, train_row)
        if name == "topsoe_incr": dist = topsoe_incr(test_row, train_row)
        if name == "topsoe_norm": dist = topsoe_norm(test_row, train_row)
        if name == "jensen_shannon": dist = jensen_shannon(test_row, train_row)
        if name == "jensen_diff": dist = jensen_diff(test_row, train_row)

        # Combinations
        if name == "taneja": dist = taneja(test_row, train_row)
        if name == "kumar_johnson": dist = kumar_johnson(test_row, train_row)
        if name == "avgL": dist = avgL(test_row, train_row)
            
        # Vicissitude
        if name == "vicis_wave_hedges": dist = vicis_wave_hedges(test_row, train_row)
        if name == "vicis_symm1": dist = vicis_symm1(test_row, train_row)
        if name == "vicis_symm2": dist = vicis_symm2(test_row, train_row)
        if name == "vicis_symm3": dist = vicis_symm3(test_row, train_row)
        if name == "min_symm": dist = min_symm(test_row, train_row)
        if name == "max_symm": dist = max_symm(test_row, train_row)
        
        distances.append((idx, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = [distances[i][0] for i in range(k)]
    return neighbors

### Knn, predict positions and calculate error

In [5]:
# calculate knn of fingerprints
def knn(k, tstrss, trnrss, distance="euclidean"):
    all_neighbors = []

    # iterate test data and find id of k nearest neighbors from trnrss
    for i in tstrss:
        all_neighbors.append(get_neighbors(trnrss, i, k, distance))

    return all_neighbors

# predict position of neighbors 
def predict_position(all_neighbors, trncrd, k):
    predicted_pos = []
    
    # iterate all neighbors to predict position. Find k indexes in trncrd and calculate mean value.
    for knn in all_neighbors:    
        x, y, z = 0, 0, 0
        for i in knn:
            x += trncrd[i][0]
            y += trncrd[i][1]
            z += trncrd[i][2]
        predicted_pos.append([x/k, y/k, round(z/k, 1)]) # mean of the error
        
    return predicted_pos

# error calculation: returns euclidean distance between predictions and tstcrd
def calculate_error(predicted_pos, tstcrd):
    error_distances = [math.dist(predicted_pos[i], tstcrd[i]) for i, val in enumerate(predicted_pos)]
    return error_distances

In [6]:
# calculates knn and returns prediction of position
def knn_and_prediction(k, filename):
    trnrss, trncrd, tstrss, tstcrd = load_Data(filename)
    neighbors = knn(k, tstrss, trnrss)
    return predict_position(neighbors, trncrd, k)

# calculates knn, predicts position and returns error distances between prediction and real position
def positioning_error(k, filename, distance="euclidean"):
    trnrss, trncrd, tstrss, tstcrd = load_Data(filename)
    neighbors = knn(k, tstrss, trnrss, distance)
    positions = predict_position(neighbors, trncrd, k)
    return calculate_error(positions, tstcrd)

In [7]:
# calculates average error with 2 decimals from list of positioning errors
def avg_error(err_list):
    return round(st.fmean(err_list), 2)

### Plots

In [8]:
#ECDF function to generate x and y axis data
def ecdf(xdata):
    xdataecdf = np.sort(xdata)
    ydataecdf = np.arange(1, len(xdata) + 1) / len(xdata)
    return xdataecdf, ydataecdf

def plot_ecdf(data1, data2, data3, title, figure):
    #Get the x and y data for ecdf plot from ecdf method
    x1,y1 = ecdf(data1)
    x5,y5 = ecdf(data2)
    x11,y11 = ecdf(data3)

    #Plot the data using matplotlib
    plt.figure(figure)
    plt.plot(x1, y1, marker = '.', linestyle = 'none', markersize = 3)
    plt.plot(x5, y5, marker = '.', linestyle = 'none', markersize = 3)
    plt.plot(x11, y11, marker = '.', linestyle = 'none', markersize = 3)
    plt.legend(("k=1", "k=5", "k=11"))
    plt.title(title)
    plt.xlabel('Error distance')
    plt.ylabel('Error probability')
    plt.margins(0.1)

### Distance metrics

In [9]:
# auxiliar functions
def normalize(h):
    return h / np.sum(h)

def square(n):
    return n*n

def half(n):
    return 0.5*n

def exp32(n):
    return n*1.5

# distance metrics
def gower(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    A=sum(abs(P-Q))
    return A/len(P)

def soergel(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    A=sum(abs(P-Q))
    return A/sum(np.maximum(P,Q))

def kulczynski_d(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    A=sum(abs(P-Q))
    return A/sum(np.minimum(P,Q))

def lorentzian(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum(np.log(1+abs(P-Q)))

def intersection(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    A=sum(abs(P-Q))
    return 0.5*A

def wavehedges(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum(abs(P-Q)/np.maximum(P,Q))

def czekanowski_s(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    A=sum(abs(P-Q))
    return A/sum(P+Q)

def czekanowski_d(P, Q):
    return 1 - czekanowski_s(P, Q)

def motyka_s(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    maxPQ=sum(np.maximum(P,Q))
    return maxPQ/sum(P+Q)

def motyka_d(P, Q):
    return 1 - motyka_s(P, Q)

def kulczynski_s(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    minPQ=sum(np.minimum(P,Q))
    A=sum(abs(P-Q))
    dist = 0
    if A != 0:
        dist = minPQ/A
    if sum(np.minimum(P,Q)) != 0:
        dist = 1/dist
    return dist

def ruzicka(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    maxPQ=sum(np.maximum(P,Q))
    return 1-sum(np.minimum(P,Q))/maxPQ
        
def tanimoto(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    maxPQ=sum(np.maximum(P,Q))
    return sum(np.maximum(P,Q)-np.minimum(P,Q))/maxPQ

def harmonic(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 1-2.*sum(P*Q/(P+Q))

def kumar(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P-Q)*(P-Q))/(sum(P*P)+sum(Q*Q)-len(P))

def jaccard_s(P, Q):
    return kumar(P, Q)
        
def jaccard_d(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 1 - jaccard_s(P, Q)

def dice_s(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P-Q)*(P-Q))/(sum(P*P)+sum(Q*Q))
    
def dice_d(P, Q):
    return 1 - dice_s(P, Q)
    
def fidelity(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum(np.sqrt(P*Q))

def bhattacharrya(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return -np.log(sum(np.sqrt(P*Q)))

def hellinger(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 2*np.sqrt(1-sum(np.sqrt(P*Q)))

def matusita(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return np.sqrt(2-2*sum(np.sqrt(P*Q)))

def squared_chord(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((np.sqrt(P)-np.sqrt(Q))**2)

def pearson(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P-Q)**2/Q)

def neyman(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P-Q)**2/P)

def squared(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P-Q)**2/(P+Q))
                
def prob_sym(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 2*sum((P-Q)**2/(P+Q))

def divergence(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 2*sum((P-Q)**2/(P+Q)**2)
    
def clark(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return np.sqrt(sum((abs(P-Q)/(P+Q))**2))

def additive_sym(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P-Q)**2*(P+Q)/(P*Q))

def kullback_PQ(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum(P*np.log(P/Q))

def kullback_QP(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum(P*np.log(Q/P))

def jeffreys(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P-Q)*np.log(P/Q))
    
def k_divergence(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum(P*np.log(2*P/(P+Q)))
    
def topsoe(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
    
def jensen_shannon(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 0.5*sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))

def jensen_diff(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 0.5*sum(P*np.log(P)+Q*np.log(Q)-(P+Q)*np.log((P+Q)/2.))

def taneja(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 0.5*sum((P+Q)*np.log((P+Q)/(2.*np.sqrt(P*Q))))

def kumar_johnson(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P*P-Q*Q)**2/(2*(P*Q)**(1.5)))

def avgL(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return 0.5*(sum(abs(P-Q))+max(abs(P-Q)))
                   
def vicis_wave_hedges(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum(abs(P-Q)/np.minimum(P,Q))
                   
def vicis_symm1(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    pqmin=np.minimum(P,Q)
    return sum((P-Q)*(P-Q)/pqmin**2)
                   
def vicis_symm2(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    pqmin=np.minimum(P,Q)
    return sum((P-Q)*(P-Q)/pqmin)
                   
def vicis_symm3(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return sum((P-Q)*(P-Q)/np.maximum(P,Q))
                   
def min_symm(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return min(P.all(),Q.all())
                   
def max_symm(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    return max(P.all(),Q.all())

In [35]:
def fidelity_abs(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    P=abs(P)
    Q=abs(Q)
    return sum(np.sqrt(P*Q))

def fidelity_incr(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    P += 100
    Q += 100
    return sum(np.sqrt(P*Q))

def fidelity_norm(P, Q):
    P=np.array(P,dtype=float).reshape(-1, 1); Q=np.array(Q,dtype=float).reshape(-1, 1)
    scaler = MinMaxScaler()
    P=scaler.fit_transform(P)
    Q=scaler.fit_transform(Q)
    return sum(np.sqrt(P*Q))

In [37]:
err = positioning_error(1, "MAN2", "fidelity_abs")
print("{}: {}".format("fidelity_abs", avg_error(err)))

err = positioning_error(1, "MAN2", "fidelity_incr")
print("{}: {}".format("fidelity_incr", avg_error(err)))

err = positioning_error(1, "MAN2", "fidelity_norm")
print("{}: {}".format("fidelity_norm", avg_error(err)))

fidelity_abs: 18.44
fidelity_incr: 16.55
fidelity_norm: 18.67


In [51]:
def bhattacharrya_abs(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    P=abs(P)
    Q=abs(Q)
    return -np.log(sum(np.sqrt(P*Q)))

def bhattacharrya_incr(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    P += 100
    Q += 100
    return -np.log(sum(np.sqrt(P*Q)))

def bhattacharrya_norm(P, Q):
    P=np.array(P,dtype=float).reshape(-1, 1); Q=np.array(Q,dtype=float).reshape(-1, 1)
    scaler = MinMaxScaler()
    P=scaler.fit_transform(P)
    Q=scaler.fit_transform(Q)
    return -np.log(sum(np.sqrt(P*Q)))

In [52]:
err = positioning_error(1, "MAN2", "bhattacharrya_abs")
print("{}: {}".format("bhattacharrya_abs", avg_error(err)))

err = positioning_error(1, "MAN2", "bhattacharrya_incr")
print("{}: {}".format("bhattacharrya_incr", avg_error(err)))

err = positioning_error(1, "MAN2", "bhattacharrya_norm")
print("{}: {}".format("bhattacharrya_norm", avg_error(err)))

bhattacharrya_abs: 13.14
bhattacharrya_incr: 10.28
bhattacharrya_norm: 11.63


In [63]:
def topsoe_abs(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    P=abs(P)
    Q=abs(Q)
    return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))

def topsoe_incr(P, Q):
    P=np.array(P,dtype=float); Q=np.array(Q,dtype=float)
    P += 100
    Q += 100
    return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))

def topsoe_norm(P, Q):
    P=np.array(P,dtype=float).reshape(-1, 1); Q=np.array(Q,dtype=float).reshape(-1, 1)
    scaler = MinMaxScaler()
    P=scaler.fit_transform(P)
    Q=scaler.fit_transform(Q)
    return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))

In [66]:
err = positioning_error(1, "MAN2", "topsoe_abs")
print("{}: {}".format("topsoe_abs", avg_error(err)))

err = positioning_error(1, "MAN2", "topsoe_incr")
print("{}: {}".format("topsoe_incr", avg_error(err)))

err = positioning_error(1, "MAN2", "topsoe_norm")
print("{}: {}".format("topsoe_norm", avg_error(err)))

topsoe_abs: 2.36
topsoe_incr: 3.21


<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in true_divide
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in true_divide
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in true_divide
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in true_divide
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in true_divide
  return sum(P*np.log(2*P/(P+Q))+Q*

<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipy

<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipy

<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython

...

<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython

<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython

<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython

<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: invalid value encountered in multiply
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython-input-63-96370724b5e1>:18: RuntimeWarning: divide by zero encountered in log
  return sum(P*np.log(2*P/(P+Q))+Q*np.log(2*Q/(P+Q)))
<ipython

KeyboardInterrupt: 